# 데이터 처리 구성 흐름도

1. 날짜, 연령-성별별 판매량 데이터 추출 (20M, 20F, 30M, 30F, 40M, 40F...)


2. 1번에서 추출한 숫자 바탕으로 연령-성별별 선호도 점수 계산


3. 카테고리 관련 원 핫 인코딩

## 1. 판매량 데이터 추출

### buy2018_1 데이터 처리 과정 
같은 과정을 이용하였기 때문에, 본 파일에서는 buy2018_1 data에 대해서만 보인다.

In [1]:
path_2018_1 = '../data/2018_1.csv'

| data | 처리 과정에서 설정한 이름 | 처리 과정에서 설정한 데이터프레임 이름 | 처리 후 저장 이름 |
| --- | --- | --- | --- |
| buy2018_1 | buy_1 | df_1 | my_buy_1 | 
| buy2018_2 | buy_2 | df_2 | my_buy_1 |
| buy2019_1 | buy_3 | df_3 | my_buy_2 |
| buy2019_2 | buy_4 | df_4 | my_buy_2 |

In [3]:
import pandas as pd

buy_1=pd.read_csv(path_2018_1)

In [4]:
buy_1

,Unnamed: 0,buy2018_1.date,buy2018_1.sex,buy2018_1.age,buy2018_1.big_cat,buy2018_1.sm_cat,buy2018_1.qty
0,1,20180101,F,20,식품,가공란,37
1,2,20180101,F,30,식품,가공란,16
2,3,20180101,F,40,식품,가공란,9
3,4,20180101,F,50,식품,가공란,3
4,5,20180101,M,20,식품,가공란,13
...,...,...,...,...,...,...,...
498005,498006,20180630,F,20,냉난방가전,히터,2
498006,498007,20180630,F,30,냉난방가전,히터,2
498007,498008,20180630,F,40,냉난방가전,히터,1
498008,498009,20180630,M,40,냉난방가전,히터,2


In [5]:
# 만들 새로운 셋을 my_buy_1 이라는 이름의 csv 파일로 만든다.
# 이를 위해 df_1이라는 새로운 dataframe을 제작한다

In [6]:
data = []
df_1 = pd.DataFrame(data, columns = ['날짜', '대분류', '소분류', '20M', '20F', '30M', '30F', '40M', '40F', '50M', '50F', '60M', '60F'])

In [7]:
df_1

,날짜,대분류,소분류,20M,20F,30M,30F,40M,40F,50M,50F,60M,60F


In [13]:
import datetime
from datetime import date

before_date = buy_1.iloc[0, 1]
after_date = datetime.datetime.strptime(str(before_date), "%Y%m%d").date()

print(str(before_date) + "에서 datetime 형식으로: " + str(after_date))

20180101에서 datetime 형식으로: 2018-01-01


한 날짜의 소분류를 한 열에 지정하기 위해 for문을 이용할 것이므로 (시간 복잡도는 O(N)이다.) count를 지정해 준다.

In [15]:
count_buy = 0 # 기존 buy2018_1 데이터의 열의 현재 위치를 보여 주는 역할
count_1 = 0 # df_1 데이터의 열의 현재 위치를 보여 주는 역할

In [17]:
df_1.loc[count_1, '소분류'] = buy_1.loc[count_buy, 'buy2018_1.sm_cat']
df_1.loc[count_1, '대분류'] = buy_1.loc[count_buy, 'buy2018_1.big_cat']
df_1.loc[count_1, '날짜'] = datetime.datetime.strptime(str(buy_1.loc[count_buy, 'buy2018_1.date']), "%Y%m%d").date()

In [18]:
df_1

,날짜,대분류,소분류,20M,20F,30M,30F,40M,40F,50M,50F,60M,60F
0,2018-01-01,식품,가공란,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


이런 식으로 저장해 준다.

In [19]:
femaleAge = {"20": "20F", "30": "30F", "40": "40F", "50": "50F", "60": "60F"}
maleAge = {"20": "20M", "30": "30M", "40": "40M", "50": "50M", "60": "60M"}

용이한 라벨링을 위해 딕셔너리를 만들어 준다.

In [21]:
max = len(buy_1.index)

In [22]:
while (True):
    if count_buy == max: break
    
    if df_1.loc[count_1, '소분류'] != buy_1.loc[count_buy, 'buy2018_1.sm_cat']:
        count_1 += 1
        df_1.loc[count_1, '소분류'] = buy_1.loc[count_buy, 'buy2018_1.sm_cat']
        df_1.loc[count_1, '대분류'] = buy_1.loc[count_buy, 'buy2018_1.big_cat']
        df_1.loc[count_1, '날짜'] = datetime.datetime.strptime(str(buy_1.loc[count_buy, 'buy2018_1.date']), "%Y%m%d").date()
        
    else:
        sex = buy_1.loc[count_buy, 'buy2018_1.sex']
        age = buy_1.loc[count_buy, 'buy2018_1.age']
        
        if sex == 'F':
            category = femaleAge[str(age)]
            df_1.loc[count_1, category] = buy_1.loc[count_buy, 'buy2018_1.qty']
            
        if sex == 'M':
            category = maleAge[str(age)]
            df_1.loc[count_1, category] = buy_1.loc[count_buy, 'buy2018_1.qty']
        
        count_buy += 1

In [23]:
df_1

,날짜,대분류,소분류,20M,20F,30M,30F,40M,40F,50M,50F,60M,60F
0,2018-01-01,식품,가공란,13,37,6,16,6,9,NaN,3,NaN,NaN
1,2018-01-01,냉난방가전,가열식 가습기,1,1,NaN,NaN,2,3,NaN,NaN,NaN,NaN
2,2018-01-01,식품,가자미,NaN,3,2,5,1,4,3,2,1,NaN
3,2018-01-01,식품,갈비/찜/바비큐용 돈육,NaN,7,7,4,1,2,NaN,1,NaN,NaN
4,2018-01-01,식품,갈비용 우육,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55288,2018-06-30,식품,환자식,NaN,2,NaN,2,1,6,NaN,7,1,1
55289,2018-06-30,식품,회,124,245,174,361,113,192,40,77,15,23
55290,2018-06-30,냉난방가전,휴대용 선풍기,14,28,18,37,21,25,9,11,2,3
55291,2018-06-30,식품,흰우유,11,30,72,185,49,106,9,27,5,2


In [ ]:
df_1.to_csv('my_buy_1.csv', index=False)

이런 형식으로 네 가지 데이터에 대해 똑같이 처리해 준다.

```python
path = '../data/2018_2.csv'

count_buy = 0
count_2 = 0


import pandas as pd
import datetime

buy_2=pd.read_csv(path)

data = []
df_2 = pd.DataFrame(data, columns = ['날짜', '대분류', '소분류', '20M', '20F', '30M', '30F', '40M', '40F', '50M', '50F', '60M', '60F'])

before_date = buy_2.loc[count_buy, 'buy2018_2.date']
df_2.loc[count_2, '소분류'] = buy_2.loc[count_buy, 'buy2018_2.sm_cat']
df_2.loc[count_2, '대분류'] = buy_2.loc[count_buy, 'buy2018_2.big_cat']
df_2.loc[count_2, '날짜'] = datetime.datetime.strptime(str(buy_2.loc[count_buy, 'buy2018_2.date']), "%Y%m%d").date()

femaleAge = {"20": "20F", "30": "30F", "40": "40F", "50": "50F", "60": "60F"}
maleAge = {"20": "20M", "30": "30M", "40": "40M", "50": "50M", "60": "60M"}

max = len(buy_2.index)

while(count_buy < max):
    if df_2.loc[count_2, '소분류'] != buy_2.loc[count_buy, 'buy2018_2.sm_cat']:
        count_1 += 1
        df_2.loc[count_2, '소분류'] = buy_2.loc[count_buy, 'buy2018_2.sm_cat']
        df_2.loc[count_2, '대분류'] = buy_2.loc[count_buy, 'buy2018_2.big_cat']
        df_2.loc[count_2, '날짜'] = datetime.datetime.strptime(str(buy_2.loc[count_buy, 'buy2018_2.date']), "%Y%m%d").date()
        
    else:
        sex = buy_2.loc[count_buy, 'buy2018_2.sex']
        age = buy_2.loc[count_buy, 'buy2018_2.age']
        
        if sex == 'F':
            category = femaleAge[str(age)]
            df_2.loc[count_2, category] = buy_2loc[count_buy, 'buy2018_2.qty']
            
        if sex == 'M':
            category = maleAge[str(age)]
            df_2.loc[count_2, category] = buy_2.loc[count_buy, 'buy2018_2.qty']
        
        count_buy += 1

df_2.to_csv('my_buy_2.csv', index=False)
```

## 2. 연령-성별별 선호도 계산

In [1]:
import pandas as pd

data_2018_1 = pd.read_csv(r'C:\Users\redem\Desktop\소비_sns_데이터/my_buy_1.csv')
data_2018_2 = pd.read_csv(r'C:\Users\redem\Desktop\소비_sns_데이터/my_buy_2.csv')
data_2019_1 = pd.read_csv(r'C:\Users\redem\Desktop\소비_sns_데이터/my_buy_3.csv')
data_2019_2 = pd.read_csv(r'C:\Users\redem\Desktop\소비_sns_데이터/my_buy_4.csv')

2018-2019 데이터를 한 데이터프레임이 놓고 처리해야 되므로 data_all로 합쳐 준다.

In [2]:
data_2018 = pd.concat([data_2018_1, data_2018_2], axis=0)
data_2019 = pd.concat([data_2019_1, data_2019_2], axis=0)
data_all = pd.concat([data_2018, data_2019], axis=0)

In [3]:
data_all

,날짜,대분류,소분류,20M,20F,30M,30F,40M,40F,50M,50F,60M,60F
0,2018-01-01,식품,가공란,13.0,37.0,6.0,16.0,6.0,9.0,NaN,3.0,NaN,NaN
1,2018-01-01,냉난방가전,가열식 가습기,1.0,1.0,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN
2,2018-01-01,식품,가자미,NaN,3.0,2.0,5.0,1.0,4.0,3.0,2.0,1.0,NaN
3,2018-01-01,식품,갈비/찜/바비큐용 돈육,NaN,7.0,7.0,4.0,1.0,2.0,NaN,1.0,NaN,NaN
4,2018-01-01,식품,갈비용 우육,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67569,2019-12-31,냉난방가전,황토매트,NaN,1.0,2.0,NaN,NaN,NaN,2.0,1.0,1.0,1.0
67570,2019-12-31,식품,회,217.0,497.0,375.0,638.0,206.0,307.0,56.0,105.0,16.0,42.0
67571,2019-12-31,냉난방가전,휴대용 선풍기,1.0,NaN,3.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN
67572,2019-12-31,식품,흰우유,37.0,66.0,98.0,253.0,77.0,153.0,24.0,49.0,14.0,11.0


선호도 점수를 구하기 위해 각 연령대가 각자 2018-2019 동안 얼마나 많이 구매했는지 구한다.

In [4]:
sum_20M = data_all['20M'].sum(axis=0)
sum_20F = data_all['20F'].sum(axis=0)

sum_30M = data_all['30M'].sum(axis=0)
sum_30F = data_all['30F'].sum(axis=0)

sum_40M = data_all['40M'].sum(axis=0)
sum_40F = data_all['40F'].sum(axis=0)

sum_50M = data_all['50M'].sum(axis=0)
sum_50F = data_all['50F'].sum(axis=0)

sum_60M = data_all['60M'].sum(axis=0)
sum_60F = data_all['60F'].sum(axis=0)

선호도를 저장하는 dataframe 을 새로 생성한다. 이는 **원 핫 인코딩된 기존 dataframe과 추후 합치도록** 한다.

선호도에 대한 dataframe 이름을 ```df_all```이라고 명명한다.

In [5]:
data = []
df_all = pd.DataFrame(data, columns = ['합계', 'sum.score', '20M.score', '20F.score', '30M.score', '30F.score', '40M.score', '40F.score', '50M.score', '50F.score', '60M.score', '60F.score'])

In [6]:
df_all

,합계,sum.score,20M.score,20F.score,30M.score,30F.score,40M.score,40F.score,50M.score,50F.score,60M.score,60F.score


In [7]:
df_all['합계'] = data_all.sum(axis=1)

In [8]:
df_all

,합계,sum.score,20M.score,20F.score,30M.score,30F.score,40M.score,40F.score,50M.score,50F.score,60M.score,60F.score
0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
67569,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67570,2459.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67571,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67572,782.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


K식 구하기 위해 계산을 진행한다.

In [9]:
df_all['20M.score'] = data_all['20M'] / sum_20M
df_all['20F.score'] = data_all['20F'] / sum_20F

df_all['30M.score'] = data_all['30M'] / sum_30M
df_all['30F.score'] = data_all['30F'] / sum_30F

df_all['40M.score'] = data_all['40M'] / sum_40M
df_all['40F.score'] = data_all['40F'] / sum_40F

df_all['50M.score'] = data_all['50M'] / sum_50M
df_all['50F.score'] = data_all['50F'] / sum_50F

df_all['60M.score'] = data_all['60M'] / sum_60M
df_all['60F.score'] = data_all['60F'] / sum_60F

In [10]:
df_all

,합계,sum.score,20M.score,20F.score,30M.score,30F.score,40M.score,40F.score,50M.score,50F.score,60M.score,60F.score
0,90.0,NaN,7.348419e-06,7.063968e-06,1.577113e-06,1.700359e-06,1.687507e-06,1.168915e-06,NaN,9.094155e-07,NaN,NaN
1,7.0,NaN,5.652630e-07,1.909181e-07,NaN,NaN,5.625025e-07,3.896383e-07,NaN,NaN,NaN,NaN
2,21.0,NaN,NaN,5.727542e-07,5.257045e-07,5.313621e-07,2.812512e-07,5.195177e-07,0.000002,6.062770e-07,0.000002,NaN
3,22.0,NaN,NaN,1.336426e-06,1.839966e-06,4.250897e-07,2.812512e-07,2.597588e-07,NaN,3.031385e-07,NaN,NaN
4,2.0,NaN,NaN,1.909181e-07,NaN,NaN,NaN,1.298794e-07,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
67569,8.0,NaN,NaN,1.909181e-07,5.257045e-07,NaN,NaN,NaN,0.000001,3.031385e-07,0.000002,0.000001
67570,2459.0,NaN,1.226621e-04,9.488628e-05,9.856958e-05,6.780180e-05,5.793775e-05,3.987298e-05,0.000032,3.182954e-05,0.000029,0.000055
67571,6.0,NaN,5.652630e-07,NaN,7.885567e-07,1.062724e-07,NaN,1.298794e-07,NaN,NaN,NaN,NaN
67572,782.0,NaN,2.091473e-05,1.260059e-05,2.575952e-05,2.688692e-05,2.165634e-05,1.987155e-05,0.000014,1.485379e-05,0.000025,0.000014


NaN 값을 0으로 바꾸어 준다.

In [11]:
df_all = df_all.fillna(0)

선호도에 대한 분모를 구하기 위해 각 물품의 그날 판매량의 대한 모든 연령대의 K값을 더한다.

In [12]:
df_all['sum.score'] = df_all['20M.score'] + df_all['20F.score'] + df_all['30M.score'] + df_all['30F.score'] + df_all['40M.score'] + df_all['40F.score'] + df_all['50M.score'] + df_all['50F.score'] + df_all['60M.score'] + df_all['60F.score']  

분자와 분모를 모두 구했으므로 기존 score에 있는 값은 분자로, sum.score의 값은 분모로 하여 최종 선호도를 계산한다.

In [13]:
df_all['20M.score'] = df_all['20M.score'] / df_all['sum.score']
df_all['20F.score'] = df_all['20F.score'] / df_all['sum.score']

df_all['30M.score'] = df_all['30M.score'] / df_all['sum.score']
df_all['30F.score'] = df_all['30F.score'] / df_all['sum.score']

df_all['40M.score'] = df_all['40M.score'] / df_all['sum.score']
df_all['40F.score'] = df_all['40F.score'] / df_all['sum.score']

df_all['50M.score'] = df_all['50M.score'] / df_all['sum.score']
df_all['50F.score'] = df_all['50F.score'] / df_all['sum.score']

df_all['60M.score'] = df_all['60M.score'] / df_all['sum.score']
df_all['60F.score'] = df_all['60F.score'] / df_all['sum.score']

In [14]:
df_all

,합계,sum.score,20M.score,20F.score,30M.score,30F.score,40M.score,40F.score,50M.score,50F.score,60M.score,60F.score
0,90.0,2.145570e-05,0.342493,0.329235,0.073506,0.079250,0.078651,0.054480,0.000000,0.042386,0.000000,0.000000
1,7.0,1.708322e-06,0.330888,0.111758,0.000000,0.000000,0.329272,0.228082,0.000000,0.000000,0.000000,0.000000
2,21.0,6.550968e-06,0.000000,0.087430,0.080248,0.081112,0.042933,0.079304,0.263728,0.092548,0.272696,0.000000
3,22.0,4.445630e-06,0.000000,0.300616,0.413882,0.095620,0.063265,0.058430,0.000000,0.068188,0.000000,0.000000
4,2.0,3.207975e-07,0.000000,0.595136,0.000000,0.000000,0.000000,0.404864,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
67569,8.0,5.273552e-06,0.000000,0.036203,0.099687,0.000000,0.000000,0.000000,0.218408,0.057483,0.338752,0.249468
67570,2459.0,6.296472e-04,0.194811,0.150698,0.156547,0.107682,0.092016,0.063326,0.051219,0.050551,0.045395,0.087755
67571,6.0,1.589972e-06,0.355518,0.000000,0.495956,0.066839,0.000000,0.081687,0.000000,0.000000,0.000000,0.000000
67572,782.0,1.958462e-04,0.106792,0.064339,0.131529,0.137286,0.110578,0.101465,0.070573,0.075844,0.127702,0.073892


## 3. 원 핫 인코딩

In [38]:
data_all = pd.get_dummies(data = data_all, columns = ['대분류'])

In [39]:
data_all = pd.get_dummies(data = data_all, columns = ['소분류'])

In [40]:
data_all

,날짜,20M,20F,30M,30F,40M,40F,50M,50F,60M,...,소분류_홍어,소분류_홍차,소분류_화장 비누,소분류_화장 퍼프,소분류_환자식,소분류_황토매트,소분류_회,소분류_휴대용 선풍기,소분류_흰우유,소분류_히터
0,2018-01-01,13.0,37.0,6.0,16.0,6.0,9.0,NaN,3.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2018-01-01,1.0,1.0,NaN,NaN,2.0,3.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2018-01-01,NaN,3.0,2.0,5.0,1.0,4.0,3.0,2.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,2018-01-01,NaN,7.0,7.0,4.0,1.0,2.0,NaN,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,2018-01-01,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67569,2019-12-31,NaN,1.0,2.0,NaN,NaN,NaN,2.0,1.0,1.0,...,0,0,0,0,0,1,0,0,0,0
67570,2019-12-31,217.0,497.0,375.0,638.0,206.0,307.0,56.0,105.0,16.0,...,0,0,0,0,0,0,1,0,0,0
67571,2019-12-31,1.0,NaN,3.0,1.0,NaN,1.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
67572,2019-12-31,37.0,66.0,98.0,253.0,77.0,153.0,24.0,49.0,14.0,...,0,0,0,0,0,0,0,0,1,0


pandas의 get_dummies로 원 핫 인코딩을 수행한다. 이때 column명의 순서를 대분류-소분류-연령별 으로 바꾸어 주기 위해 아래 내용을 수행한다.

In [41]:
col1 = data_all.columns[1:10].to_list() # 구매량
col2 = data_all.columns[11:].to_list() # 대분류 + 소분류
col3 = ['날짜']

data_all = data_all[col3 + col2 + col1] # column명의 순서를 날짜 + 대분류 + 소분류 + 구매량으로 바꾼다.

In [42]:
data_all

,날짜,대분류_냉난방가전,대분류_뷰티,대분류_식품,소분류_가공란,소분류_가스온수기,소분류_가열식 가습기,소분류_가자미,소분류_갈비/찜/바비큐용 돈육,소분류_갈비용 우육,...,소분류_히터,20M,20F,30M,30F,40M,40F,50M,50F,60M
0,2018-01-01,0,0,1,1,0,0,0,0,0,...,0,13.0,37.0,6.0,16.0,6.0,9.0,NaN,3.0,NaN
1,2018-01-01,1,0,0,0,0,1,0,0,0,...,0,1.0,1.0,NaN,NaN,2.0,3.0,NaN,NaN,NaN
2,2018-01-01,0,0,1,0,0,0,1,0,0,...,0,NaN,3.0,2.0,5.0,1.0,4.0,3.0,2.0,1.0
3,2018-01-01,0,0,1,0,0,0,0,1,0,...,0,NaN,7.0,7.0,4.0,1.0,2.0,NaN,1.0,NaN
4,2018-01-01,0,0,1,0,0,0,0,0,1,...,0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67569,2019-12-31,1,0,0,0,0,0,0,0,0,...,0,NaN,1.0,2.0,NaN,NaN,NaN,2.0,1.0,1.0
67570,2019-12-31,0,0,1,0,0,0,0,0,0,...,0,217.0,497.0,375.0,638.0,206.0,307.0,56.0,105.0,16.0
67571,2019-12-31,1,0,0,0,0,0,0,0,0,...,0,1.0,NaN,3.0,1.0,NaN,1.0,NaN,NaN,NaN
67572,2019-12-31,0,0,1,0,0,0,0,0,0,...,0,37.0,66.0,98.0,253.0,77.0,153.0,24.0,49.0,14.0


확인 결과 올바르게 수정되었다.

NaN 값을 0으로 바꾸어 준다.

In [43]:
data_all = data_all.fillna(0)

## 4. 마지막 데이터 프레임 합치기, 라벨링 처리

앞서 만들었던 선호도 점수와 원 핫 인코딩된 기존 데이터프레임을 합친다. 이를 ```final_data```라고 명명한다.

In [15]:
final_data = pd.concat([data_all, df_all], axis=1)

합친 후 라벨링 처리를 기획안대로 해 준다.

In [16]:
final_data.rename(columns = {'20M' : '20대 남성 판매량(개)'}, inplace = True)
final_data.rename(columns = {'20F' : '20대 여성 판매량(개)'}, inplace = True)

final_data.rename(columns = {'30M' : '30대 남성 판매량(개)'}, inplace = True)
final_data.rename(columns = {'30F' : '30대 여성 판매량(개)'}, inplace = True)

final_data.rename(columns = {'40M' : '40대 남성 판매량(개)'}, inplace = True)
final_data.rename(columns = {'40F' : '40대 여성 판매량(개)'}, inplace = True)

final_data.rename(columns = {'50M' : '50대 남성 판매량(개)'}, inplace = True)
final_data.rename(columns = {'50F' : '50대 여성 판매량(개)'}, inplace = True)

final_data.rename(columns = {'60M' : '60대 남성 판매량(개)'}, inplace = True)
final_data.rename(columns = {'60F' : '60대 여성 판매량(개)'}, inplace = True)

In [17]:
final_data.rename(columns = {'20M.score' : '20대 남성 선호도 점수'}, inplace = True)
final_data.rename(columns = {'20F.score' : '20대 여성 선호도 점수'}, inplace = True)

final_data.rename(columns = {'30M.score' : '30대 남성 선호도 점수'}, inplace = True)
final_data.rename(columns = {'30F.score' : '30대 여성 선호도 점수'}, inplace = True)

final_data.rename(columns = {'40M.score' : '40대 남성 선호도 점수'}, inplace = True)
final_data.rename(columns = {'40F.score' : '40대 여성 선호도 점수'}, inplace = True)

final_data.rename(columns = {'50M.score' : '50대 남성 선호도 점수'}, inplace = True)
final_data.rename(columns = {'50F.score' : '50대 여성 선호도 점수'}, inplace = True)

final_data.rename(columns = {'60M.score' : '60대 남성 선호도 점수'}, inplace = True)
final_data.rename(columns = {'60F.score' : '60대 여성 선호도 점수'}, inplace = True)

In [18]:
final_data.rename(columns = {'sum.score' : '일별 물품 선호도 점수'}, inplace = True)

In [19]:
final_data.rename(columns = {'합계' : '일별 판매 합계량(개)'}, inplace = True)

In [20]:
final_data

,날짜,대분류,소분류,20대 남성 판매량(개),20대 여성 판매량(개),30대 남성 판매량(개),30대 여성 판매량(개),40대 남성 판매량(개),40대 여성 판매량(개),50대 남성 판매량(개),...,20대 남성 선호도 점수,20대 여성 선호도 점수,30대 남성 선호도 점수,30대 여성 선호도 점수,40대 남성 선호도 점수,40대 여성 선호도 점수,50대 남성 선호도 점수,50대 여성 선호도 점수,60대 남성 선호도 점수,60대 여성 선호도 점수
0,2018-01-01,식품,가공란,13.0,37.0,6.0,16.0,6.0,9.0,NaN,...,0.342493,0.329235,0.073506,0.079250,0.078651,0.054480,0.000000,0.042386,0.000000,0.000000
1,2018-01-01,냉난방가전,가열식 가습기,1.0,1.0,NaN,NaN,2.0,3.0,NaN,...,0.330888,0.111758,0.000000,0.000000,0.329272,0.228082,0.000000,0.000000,0.000000,0.000000
2,2018-01-01,식품,가자미,NaN,3.0,2.0,5.0,1.0,4.0,3.0,...,0.000000,0.087430,0.080248,0.081112,0.042933,0.079304,0.263728,0.092548,0.272696,0.000000
3,2018-01-01,식품,갈비/찜/바비큐용 돈육,NaN,7.0,7.0,4.0,1.0,2.0,NaN,...,0.000000,0.300616,0.413882,0.095620,0.063265,0.058430,0.000000,0.068188,0.000000,0.000000
4,2018-01-01,식품,갈비용 우육,NaN,1.0,NaN,NaN,NaN,1.0,NaN,...,0.000000,0.595136,0.000000,0.000000,0.000000,0.404864,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67569,2019-12-31,냉난방가전,황토매트,NaN,1.0,2.0,NaN,NaN,NaN,2.0,...,0.000000,0.036203,0.099687,0.000000,0.000000,0.000000,0.218408,0.057483,0.338752,0.249468
67570,2019-12-31,식품,회,217.0,497.0,375.0,638.0,206.0,307.0,56.0,...,0.194811,0.150698,0.156547,0.107682,0.092016,0.063326,0.051219,0.050551,0.045395,0.087755
67571,2019-12-31,냉난방가전,휴대용 선풍기,1.0,NaN,3.0,1.0,NaN,1.0,NaN,...,0.355518,0.000000,0.495956,0.066839,0.000000,0.081687,0.000000,0.000000,0.000000,0.000000
67572,2019-12-31,식품,흰우유,37.0,66.0,98.0,253.0,77.0,153.0,24.0,...,0.106792,0.064339,0.131529,0.137286,0.110578,0.101465,0.070573,0.075844,0.127702,0.073892


In [21]:
final_data = final_data.fillna(0)

In [22]:
final_data

,날짜,대분류,소분류,20대 남성 판매량(개),20대 여성 판매량(개),30대 남성 판매량(개),30대 여성 판매량(개),40대 남성 판매량(개),40대 여성 판매량(개),50대 남성 판매량(개),...,20대 남성 선호도 점수,20대 여성 선호도 점수,30대 남성 선호도 점수,30대 여성 선호도 점수,40대 남성 선호도 점수,40대 여성 선호도 점수,50대 남성 선호도 점수,50대 여성 선호도 점수,60대 남성 선호도 점수,60대 여성 선호도 점수
0,2018-01-01,식품,가공란,13.0,37.0,6.0,16.0,6.0,9.0,0.0,...,0.342493,0.329235,0.073506,0.079250,0.078651,0.054480,0.000000,0.042386,0.000000,0.000000
1,2018-01-01,냉난방가전,가열식 가습기,1.0,1.0,0.0,0.0,2.0,3.0,0.0,...,0.330888,0.111758,0.000000,0.000000,0.329272,0.228082,0.000000,0.000000,0.000000,0.000000
2,2018-01-01,식품,가자미,0.0,3.0,2.0,5.0,1.0,4.0,3.0,...,0.000000,0.087430,0.080248,0.081112,0.042933,0.079304,0.263728,0.092548,0.272696,0.000000
3,2018-01-01,식품,갈비/찜/바비큐용 돈육,0.0,7.0,7.0,4.0,1.0,2.0,0.0,...,0.000000,0.300616,0.413882,0.095620,0.063265,0.058430,0.000000,0.068188,0.000000,0.000000
4,2018-01-01,식품,갈비용 우육,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.595136,0.000000,0.000000,0.000000,0.404864,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67569,2019-12-31,냉난방가전,황토매트,0.0,1.0,2.0,0.0,0.0,0.0,2.0,...,0.000000,0.036203,0.099687,0.000000,0.000000,0.000000,0.218408,0.057483,0.338752,0.249468
67570,2019-12-31,식품,회,217.0,497.0,375.0,638.0,206.0,307.0,56.0,...,0.194811,0.150698,0.156547,0.107682,0.092016,0.063326,0.051219,0.050551,0.045395,0.087755
67571,2019-12-31,냉난방가전,휴대용 선풍기,1.0,0.0,3.0,1.0,0.0,1.0,0.0,...,0.355518,0.000000,0.495956,0.066839,0.000000,0.081687,0.000000,0.000000,0.000000,0.000000
67572,2019-12-31,식품,흰우유,37.0,66.0,98.0,253.0,77.0,153.0,24.0,...,0.106792,0.064339,0.131529,0.137286,0.110578,0.101465,0.070573,0.075844,0.127702,0.073892


In [25]:
final_data.to_csv(r'C:\Users\redem\Desktop\소비_sns_데이터\final_data_columns_no_ohe.csv', index=False)